<a href="https://colab.research.google.com/github/hukim1112/one-day-DL/blob/main/5_(optional)wandb_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. WandB 임포트

pip install wandb로 설치 후 import

In [ ]:
import wandb

### 2. WandB 초기화

프로젝트를 시작할 때 wandb.init()을 사용해 프로젝트를 초기화합니다.

In [ ]:

# 프로젝트 초기화
wandb.init(
    project="my_wandb_tutorial", #프로젝트 명칭
    name="add_logs+tables+images", #wandb에서는 프로젝트 내 실험들을 run이라고 지칭합니다.
    config={
        "learning_rate": 0.001,
        "batch_size": 32,
        "epochs": 5,
    }
)

In [ ]:
wandb.config

In [ ]:
# wandb.config 값 사용하기
learning_rate = wandb.config.learning_rate  # 또는 wandb.config["learning_rate"]
batch_size = wandb.config.batch_size
epochs = wandb.config.epochs

print(f"Learning rate: {learning_rate}, Batch size: {batch_size}, Epochs: {epochs}")


### 3. 학습 중 로그 저장

학습 중에 손실 값, 정확도와 같은 로그를 저장할 수 있습니다.

In [ ]:
import random

# 가짜 학습 예시
epochs = wandb.config.epochs
for epoch in range(epochs):
    loss = random.random()  # 예시로 랜덤 값 사용
    accuracy = random.random()

    # 로그 저장
    wandb.log({"epoch": epoch, "random_loss": loss, "random_accuracy": accuracy})


### 4. WandB 테이블 생성 및 저장

테이블을 사용하여 데이터셋, 예측 결과 등을 시각화할 수 있습니다.

In [ ]:
# WandB 테이블 생성
columns = ["image_id", "label", "prediction"]
data = [
    ["img_001", "cat", "cat"],
    ["img_002", "dog", "cat"],  # 예측 오류
    ["img_003", "bird", "bird"],
]

# 테이블 생성 및 저장
table = wandb.Table(columns=columns, data=data)
wandb.log({"prediction_table": table})


### 5. 이미지를 저장

이미지를 저장하여 시각화에 활용할 수 있습니다.

In [ ]:
from PIL import Image
import numpy as np

# 랜덤 이미지 생성
image_data = np.random.randint(0, 255, (100, 100, 3), dtype=np.uint8)
image = Image.fromarray(image_data)

# 이미지를 WandB에 저장
wandb.log({"random_image": wandb.Image(image, caption="Random Example Image")})


### 6. 학습 로그 저장

wandb를 활용하여 학습 로그를 저장하는 간단한 PyTorch 예제 코드입니다. 이 코드는 MNIST 데이터셋을 사용하여 간단한 신경망을 학습하면서 wandb에 손실(loss)과 정확도(accuracy)를 기록합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# 데이터셋 로딩
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle=True)

# 간단한 신경망 모델 정의
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 및 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

# 학습 루프
for epoch in range(wandb.config.epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # wandb 로그 저장
        if i % 100 == 0:  # 100배치마다 로깅
            wandb.log({"loss": running_loss / (i+1), "accuracy": correct / total})

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader):.4f}, Accuracy: {correct / total:.4f}")
    wandb.log({"epoch_loss": running_loss / len(trainloader), "epoch_accuracy": correct / total})

### 6. WandB 종료

모든 작업이 끝나면 wandb.finish()로 WandB 세션을 종료합니다.

In [ ]:
# WandB 세션 종료
wandb.finish()
